In [1]:
import pandas as pd
import os
import datetime as dt
import zipfile
import tempfile # for temporary file storage to avoid storing large datasets on github

In [2]:
base_dir = os.path.dirname(os.path.abspath("..\data"))
path = os.path.join(base_dir, r"data")
os.chdir(path)

In [3]:
# os.listdir()

#### Functions

In [4]:
# read zips from repo, extract data into temp dir, create dataframes
def zip_to_df(file_name):
    # create temporary directory  
    with tempfile.TemporaryDirectory() as tmpdirname:
        # collect data from zip file 
        with zipfile.ZipFile(file_name+".zip") as zip_file:
            file_list = zip_file.namelist()
            for file in file_list:
               zip_file.extract(file, tmpdirname)
            file_path = tmpdirname+"\\"+file_name
            # create and return dataframe
            df = pd.read_csv(file_path)
            return df

#### Total Demand

In [5]:
file_name = "totaldemand_nsw.csv"
totalDemand = zip_to_df(file_name)
totalDemand.head()

,DATETIME,REGIONID,TOTALDEMAND
0,2010-01-01 00:00:00,NSW1,7997.41
1,2010-01-01 00:05:00,NSW1,7946.51
2,2010-01-01 00:10:00,NSW1,7921.55
3,2010-01-01 00:15:00,NSW1,7837.84
4,2010-01-01 00:20:00,NSW1,7781.11


In [6]:
#update datatypes
totalDemand['DATETIME'] = pd.to_datetime(totalDemand['DATETIME'])
totalDemand.dtypes

DATETIME       datetime64[ns]
REGIONID               object
TOTALDEMAND           float64
dtype: object

#### Temperature

In [7]:
file_name = "temperature_nsw.csv"
temperature = zip_to_df(file_name)
temperature.head()

,DATETIME,LOCATION,TEMPERATURE
0,2010-01-01 00:00:00,94766.0,23.1
1,2010-01-01 00:01:00,94766.0,23.1
2,2010-01-01 00:30:00,94766.0,22.9
3,2010-01-01 00:50:00,94766.0,22.7
4,2010-01-01 01:00:00,94766.0,22.6


In [8]:
#update datatypes
temperature['DATETIME'] = pd.to_datetime(temperature['DATETIME'])
temperature.dtypes

DATETIME       datetime64[ns]
LOCATION              float64
TEMPERATURE           float64
dtype: object

#### Forecast Demand

In [9]:
file_name = "forecastdemand_nsw.csv"
forecastDemand = zip_to_df(file_name)
forecastDemand.head()

,DATETIME,REGIONID,FORECASTDEMAND,PREDISPATCHSEQNO,PERIODID,LASTCHANGED
0,2010-01-01 00:00:00,NSW1,7832.04,2009123018,71,2009-12-30 12:31:49
1,2010-01-01 00:00:00,NSW1,7832.04,2009123019,70,2009-12-30 13:01:43
2,2010-01-01 00:00:00,NSW1,7832.03,2009123020,69,2009-12-30 13:31:36
3,2010-01-01 00:00:00,NSW1,7832.03,2009123021,68,2009-12-30 14:01:44
4,2010-01-01 00:00:00,NSW1,7830.96,2009123022,67,2009-12-30 14:31:35


In [10]:
#update datatypes
forecastDemand['DATETIME'] = pd.to_datetime(forecastDemand['DATETIME'])
forecastDemand['LASTCHANGED'] = pd.to_datetime(forecastDemand['LASTCHANGED'])
forecastDemand.dtypes

DATETIME            datetime64[ns]
REGIONID                    object
FORECASTDEMAND             float64
PREDISPATCHSEQNO             int64
PERIODID                     int64
LASTCHANGED         datetime64[ns]
dtype: object

#### Energy Prices and Demand

In [11]:
#merge all files in folder
wd = path + r"\Price and Demand - 2010-22"
files = os.listdir(wd)
prices = pd.DataFrame([])
file_count = 0
for file in files:
    if file.endswith(".csv"):
        file_df = pd.read_csv(wd + "\\" + file)
        prices = prices.append(file_df, ignore_index=True)
        file_count += 1
print("Merged", file_count, "files")
#remove additional headers 
prices = prices[prices["REGION"]!="REGION"]
prices.head()

Merged 153 files


,REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE
0,NSW1,2010/01/01 00:30:00,7809.31,21.5,TRADE
1,NSW1,2010/01/01 01:00:00,7483.69,20.98,TRADE
2,NSW1,2010/01/01 01:30:00,7117.23,20.06,TRADE
3,NSW1,2010/01/01 02:00:00,6812.03,17.93,TRADE
4,NSW1,2010/01/01 02:30:00,6544.33,17.43,TRADE


In [12]:
#update datatypes
prices['SETTLEMENTDATE'] = pd.to_datetime(prices['SETTLEMENTDATE'])
prices['TOTALDEMAND'] = prices['TOTALDEMAND'].astype(float)
prices['RRP'] = prices['RRP'].astype(float)
prices.dtypes

REGION                    object
SETTLEMENTDATE    datetime64[ns]
TOTALDEMAND              float64
RRP                      float64
PERIODTYPE                object
dtype: object

#### BOM

In [32]:
# timeframe restriction
year_start = 2010

In [33]:
file_path = path + r"\Solar Exposure - Bankstown Airport\IDCJAC0016_066137_1800_Data.csv"
bom_solar = pd.read_csv(file_path)
bom_solar = bom_solar[bom_solar.Year >= year_start]
bom_solar.head()

,Product code,Bureau of Meteorology station number,Year,Month,Day,Daily global solar exposure (MJ/m*m)
7305,IDCJAC0016,66137,2010,1,1,14.6
7306,IDCJAC0016,66137,2010,1,2,18.4
7307,IDCJAC0016,66137,2010,1,3,7.3
7308,IDCJAC0016,66137,2010,1,4,14.4
7309,IDCJAC0016,66137,2010,1,5,28.1


In [34]:
bom_solar.dtypes

Product code                             object
Bureau of Meteorology station number      int64
Year                                      int64
Month                                     int64
Day                                       int64
Daily global solar exposure (MJ/m*m)    float64
dtype: object

In [35]:
file_path = path + r"\Temperature - Bankstown Airport\IDCJAC0010_066137_1800_Data.csv"
bom_temp = pd.read_csv(file_path)
bom_temp = bom_temp[bom_temp.Year >= year_start]
bom_temp.head()

,Product code,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality
15341,IDCJAC0010,66137,2010,1,1,29.6,1.0,Y
15342,IDCJAC0010,66137,2010,1,2,29.5,1.0,Y
15343,IDCJAC0010,66137,2010,1,3,21.0,1.0,Y
15344,IDCJAC0010,66137,2010,1,4,24.0,1.0,Y
15345,IDCJAC0010,66137,2010,1,5,30.4,1.0,Y


In [36]:
bom_temp.dtypes

Product code                                    object
Bureau of Meteorology station number             int64
Year                                             int64
Month                                            int64
Day                                              int64
Maximum temperature (Degree C)                 float64
Days of accumulation of maximum temperature    float64
Quality                                         object
dtype: object

In [37]:
file_path = path + r"\Rainfall - Bankstown\IDCJAC0009_066137_1800_Data.csv"
bom_rain = pd.read_csv(file_path)
bom_rain = bom_rain[bom_rain.Year >= year_start]
bom_rain.head()

,Product code,Bureau of Meteorology station number,Year,Month,Day,Rainfall amount (millimetres),Period over which rainfall was measured (days),Quality
15341,IDCJAC0009,66137,2010,1,1,0.2,1.0,N
15342,IDCJAC0009,66137,2010,1,2,0.0,1.0,N
15343,IDCJAC0009,66137,2010,1,3,15.2,1.0,N
15344,IDCJAC0009,66137,2010,1,4,0.2,1.0,N
15345,IDCJAC0009,66137,2010,1,5,0.0,1.0,N


In [26]:
bom_rain.dtypes

Product code                                       object
Bureau of Meteorology station number                int64
Year                                                int64
Month                                               int64
Day                                                 int64
Rainfall amount (millimetres)                     float64
Period over which rainfall was measured (days)    float64
Quality                                            object
dtype: object

In [38]:
bom_rain.Quality.unique()

array(['N', nan, 'Y'], dtype=object)

#### Calculated Columns

In [13]:
totalDemand["DATE"] = totalDemand["DATETIME"].dt.date
forecastDemand["DATE"] = forecastDemand["DATETIME"].dt.date
temperature["DATE"] = temperature["DATETIME"].dt.date
prices["DATE"] = prices["SETTLEMENTDATE"].dt.date

#### Remove redundant columns

In [14]:
totalDemand = totalDemand.drop(columns=["REGIONID"])
forecastDemand = forecastDemand.drop(columns=["REGIONID"])
temperature = temperature.drop(columns=["LOCATION"])
prices = prices.drop(columns=["REGION","PERIODTYPE"])

#### Merge Datasets

Need to look at aggregating datasets on DATE
* totalDemand: aggregate every X number of hours (DATETIME), each aggregate as a column (e.g. 00-06, 06-12, 12-18, 18-00)
* forecastDemand: aggregate every X number of hours (LAST CHANGED), each aggregate as a column (see above)
* temperature: aggregate every X number of hours (DATETIME), each aggregate as a column (see above)
* prices: aggregate every X number of hours (SETTLEMENTDATE), each aggregate as a column (see above)

In [15]:
# df = forecastDemand.merge(temperature, on='DATE')
# df = df.merge(totalDemand, on='DATE')
# df = df.merge(prices, on="DATE")
# df 